This is a cleaner file with just the conclusions draw from the testing/exploritory file

In [1]:
from flask import Flask, render_template
from sqlalchemy import create_engine
import rds_config
import pandas as pd
engine = create_engine('email for link')

    # take dates only from Feb 2021, skewing graph with null dates
sql = f"""SELECT availability.number, availability.available_bike_stands, 
    availability.available_bikes, weather.weather_id, weather.weather_main, weather.main_temp, weather.main_feels_like, weather.main_temp_max, weather.main_temp_min, weather.main_pressure, weather.main_humidity, weather.wind_speed, 
    from_unixtime(round(unix_timestamp(from_unixtime(availability.last_update/1000))/(60*15))*(60*15)) as test1 
    FROM bikes.availability, bikes.weather 
    where from_unixtime(round(unix_timestamp(from_unixtime(availability.last_update/1000))/(60*15))*(60*15))
    = from_unixtime(round(unix_timestamp(from_unixtime(weather.dt))/(60*15))*(60*15))
    
    
"""
#and weather.weather_description = (select weather_description from dublinbikes.weather
#where dt = (select max(weather.dt) from dublinbikes.weather))

eoindf = pd.read_sql(sql, engine)
#eoindf.to_csv('eoindf.csv', index=False)
#eoindf = pd.read_csv('eoindf.csv')

In [2]:
#import pandas as pd
#eoindf = pd.read_csv('eoindf.csv')

In [3]:
eoindf

,number,available_bike_stands,available_bikes,weather_id,weather_main,main_temp,main_feels_like,main_temp_max,main_temp_min,main_pressure,main_humidity,wind_speed,test1
0,2,11,9,803,Clouds,280.66,277.47,280.93,280.15,1019,1019,3.09,2021-03-08 23:15:00
1,2,11,9,803,Clouds,280.75,277.58,281.48,280.15,1019,1019,3.09,2021-03-08 23:30:00
2,2,11,9,803,Clouds,280.50,276.92,280.93,279.82,1019,1019,3.60,2021-03-09 00:45:00
3,2,16,4,803,Clouds,282.57,279.56,283.71,280.93,1016,1016,3.09,2021-03-09 10:00:00
4,2,16,4,803,Clouds,282.58,279.57,283.71,280.93,1016,1016,3.09,2021-03-09 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887817,117,35,5,803,Clouds,277.98,273.37,279.26,277.04,1026,1026,7.20,2021-04-06 10:45:00
887818,117,35,5,803,Clouds,277.98,273.37,279.26,277.04,1026,1026,7.20,2021-04-06 10:45:00
887819,117,35,5,803,Clouds,277.93,272.96,278.15,277.04,1025,1025,8.23,2021-04-06 10:45:00
887820,117,35,5,803,Clouds,277.98,273.37,279.26,277.04,1026,1026,7.20,2021-04-06 10:45:00


In [4]:
eoindf.dtypes

number                            int64
available_bike_stands             int64
available_bikes                   int64
weather_id                        int64
weather_main                     object
main_temp                       float64
main_feels_like                 float64
main_temp_max                   float64
main_temp_min                   float64
main_pressure                     int64
main_humidity                     int64
wind_speed                      float64
test1                    datetime64[ns]
dtype: object

In [2]:
eoindf['weather_main'] = eoindf['weather_main'].astype('category')

#found that doesnt have a big impact on the prediction which is strange
eoindf["weather_main"] = pd.Categorical(eoindf["weather_main"]).codes 

eoindf['hour'] = 0
eoindf['day'] = 0
for i in range(eoindf.shape[0]):
    eoindf.hour[i] = eoindf.test1[i].hour
    eoindf.day[i] = eoindf.test1[i].weekday() 
    if (i%10000==0):
        print(i)

<ipython-input-2-cde499fef10c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eoindf.hour[i] = eoindf.test1[i].hour
<ipython-input-2-cde499fef10c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eoindf.day[i] = eoindf.test1[i].weekday()


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000


In [3]:
eoindf.to_csv('eoindf2.csv', index=False)

In [9]:
X = eoindf[['number','day', 'hour', 'wind_speed', 'main_temp']]
#X = eoindf[['day', 'hour', 'wind_speed', 'main_temp']]
y = eoindf.available_bikes

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print("Training data:\n", pd.concat([X_train, y_train], axis=1))
print("\nTest data:\n", pd.concat([X_test, y_test], axis=1))

Training data:
         number  day  hour  wind_speed  main_temp  available_bikes
577402       2    6     7        6.69     284.83                0
248713     114    2    21        6.17     283.03                7
79668       38    3    23        6.69     277.74               14
531450      21    0     7        5.14     276.19                7
86922       41    4     4        5.14     277.57                5
...        ...  ...   ...         ...        ...              ...
882371     105    6    10        4.63     284.93                9
152315      71    0    23        7.20     283.43                8
117952      55    4    17        6.17     280.50               10
435829      80    1    19        2.57     283.33               10
305711      23    4     8        5.14     279.23                5

[621600 rows x 6 columns]

Test data:
         number  day  hour  wind_speed  main_temp  available_bikes
447216      86    4    17        6.17     278.18                1
669822      43    5 

In [12]:
model = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=42)
model = model.fit(X_train,y_train)


In [13]:
model.oob_score_

0.9740846159068666

In [14]:
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [15]:
test_prediction = model.predict(X_test)
printMetrics(y_test, test_prediction)


MAE:  0.4973716991915119
RMSE:  1.0763374717585097
R2:  0.9748744701735259


In [18]:
features2 = ['number', 'day', 'hour', 'wind_speed', 'main_temp']
#features2 = ['day', 'hour', 'wind_speed', 'main_temp']
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = features2,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances


,importance
number,0.446644
main_temp,0.187855
hour,0.133388
day,0.131540
wind_speed,0.100572


In [17]:
#print("\nPrediction for first example", X_test['hour'].values[0], y_test.values[0], test_prediction[0])
actual_vs_predicted = pd.concat([y_test, pd.DataFrame(test_prediction, columns=['Predicted'], index=y_test.index)], axis=1)
actual_vs_predicted.head(50)

,available_bikes,Predicted
447216,1,1.345036
669822,11,11.010000
347109,0,0.000000
196026,5,5.040000
688440,2,2.040000
454377,6,5.996000
409794,30,30.000000
174884,10,10.048492
45632,4,4.080000
670508,19,19.665786


In [19]:
model.predict(X_test)

array([ 1.34503571, 11.01      ,  0.        , ..., 32.        ,
        0.        ,  8.01      ])

In [15]:
test_prediction = model.predict(X_test)
printMetrics(y_test, test_prediction)


MAE:  0.4873122435240516
RMSE:  0.9364470285704382
R2:  0.9531480323292573


In [16]:
X_test

,day,hour,wind_speed,main_temp
5495,3,1,4.63,283.36
1916,7,24,7.72,282.32
2891,2,3,3.60,281.08
2265,2,1,9.77,283.15
5164,2,4,6.17,283.24
...,...,...,...,...
962,5,7,4.63,275.93
5690,3,15,3.09,287.24
3849,4,16,7.72,284.82
7050,7,8,3.60,277.32


In [17]:
d = {'day': [7], 'hour': [18],'wind_speed': [8.75], 'main_temp': [286.48], }
created = pd.DataFrame(data=d)
created
model.predict(created)

array([6.75])

In [18]:
output=model.predict(created)

In [19]:
output[0]

6.75

In [20]:
import pickle

In [21]:
with open ('model_pickle', 'wb') as f:
    pickle.dump(model, f)

In [22]:
with open ('model_pickle', 'rb') as f:
    mp=pickle.load(f)

In [23]:
mp.predict(created)[0]

6.75

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
mp = pickle.load(open('finalized_model.sav', 'rb'))
ans = mp.predict(created)[0]
print(ans)

In [ ]:
import joblib
filename = 'finalized_model.sav'
joblib.dump(model, filename)
 
# some time later...
 
# load the model from disk
mp = joblib.load(filename)
ans = mp.predict(created)[0]
print(ans)

#Just one station

In [20]:
from flask import Flask, render_template
from sqlalchemy import create_engine
import rds_config
import pandas as pd
engine = create_engine('email for link')

    # take dates only from Feb 2021, skewing graph with null dates
sql = f"""SELECT availability.number, availability.available_bike_stands, 
    availability.available_bikes, weather.weather_id, weather.weather_main, weather.main_temp, weather.main_feels_like, weather.main_temp_max, weather.main_temp_min, weather.main_pressure, weather.main_humidity, weather.wind_speed, 
    from_unixtime(round(unix_timestamp(from_unixtime(availability.last_update/1000))/(60*15))*(60*15)) as test1 
    FROM bikes.availability, bikes.weather 
    where from_unixtime(round(unix_timestamp(from_unixtime(availability.last_update/1000))/(60*15))*(60*15))
    = from_unixtime(round(unix_timestamp(from_unixtime(weather.dt))/(60*15))*(60*15))
    and availability.number = 2
    
"""
#and weather.weather_description = (select weather_description from dublinbikes.weather
#where dt = (select max(weather.dt) from dublinbikes.weather))

Onestation = pd.read_sql(sql, engine)

I just connected to the datebase to get the but you can use to just get the number of the station from the main csv file

filt = (eoindf['number'] == 2)
eoindf.loc[filt]

In [ ]:
filt = (eoindf['number'] == 2)
eoindf.loc[filt]

In [21]:
Onestation['weather_main'] = Onestation['weather_main'].astype('category')

#found that doesnt have a big impact on the prediction which is strange
Onestation["weather_main"] = pd.Categorical(Onestation["weather_main"]).codes 

Onestation['hour'] = 0
Onestation['day'] = 0
for i in range(Onestation.shape[0]):
    Onestation.hour[i] = Onestation.test1[i].hour
    Onestation.day[i] = Onestation.test1[i].weekday() 
    #if (i%10000==0):
     #   print(i)

<ipython-input-21-f13b81526406>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Onestation.hour[i] = Onestation.test1[i].hour
<ipython-input-21-f13b81526406>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Onestation.day[i] = Onestation.test1[i].weekday()


In [3]:
#eoindf.to_csv('eoindf2.csv', index=False)

In [22]:
#X = eoindf[['number','day', 'hour', 'wind_speed', 'main_temp']]
X = Onestation[['day', 'hour', 'wind_speed', 'main_temp']]
y = Onestation.available_bikes

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print("Training data:\n", pd.concat([X_train, y_train], axis=1))
print("\nTest data:\n", pd.concat([X_test, y_test], axis=1))

Training data:
       day  hour  wind_speed  main_temp  available_bikes
4282    4    19       11.83     277.34               11
2126    0    15        2.57     284.89                9
282     2     5        2.57     281.16                4
6418    4    13        4.63     281.57                4
6260    4     3        4.12     278.85               11
...   ...   ...         ...        ...              ...
4931    0    13        7.72     289.41                9
3264    2     2        3.60     279.54                4
1653    6     4        6.17     277.60                9
2607    1    22        4.12     280.30                4
2732    2     8        3.09     278.56                3

[5544 rows x 5 columns]

Test data:
       day  hour  wind_speed  main_temp  available_bikes
2755    2    10        7.20     281.76                3
1891    6    21        8.75     282.54               16
6745    5    11        3.09     283.09                4
949     4     5        5.14     276.12            

In [24]:
model = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=42)
model = model.fit(X_train,y_train)


In [25]:
model.oob_score_

0.9493780210577327

# Predictions dont seem to be massively greated than if we just use one pickle or joblib file

In [27]:
test_prediction = model.predict(X_test)
printMetrics(y_test, test_prediction)


MAE:  0.4942280690118307
RMSE:  0.9924021617870435
R2:  0.945920426608088


In [29]:
#features2 = ['number', 'day', 'hour', 'wind_speed', 'main_temp']
features2 = ['day', 'hour', 'wind_speed', 'main_temp']
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = features2,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances


,importance
main_temp,0.356178
hour,0.272981
day,0.200281
wind_speed,0.170561


In [30]:
#print("\nPrediction for first example", X_test['hour'].values[0], y_test.values[0], test_prediction[0])
actual_vs_predicted = pd.concat([y_test, pd.DataFrame(test_prediction, columns=['Predicted'], index=y_test.index)], axis=1)
actual_vs_predicted.head(50)

,available_bikes,Predicted
2755,3,4.283000
1891,16,15.850000
6745,4,4.075500
949,7,7.000000
1843,16,15.900000
3728,1,0.054167
244,4,4.800000
1373,0,0.060000
4065,1,1.000000
1278,2,2.000000
